In [5]:
import jupyter
from tweet_browser_test import tweet_browser as tb
import ipywidgets as widgets
import voila
from IPython.display import display
import time
import threading

###### Settings #########
fileName = "allCensus_sample.csv"
colHeaders = ["SenderScreenName", "CreatedTime", "Message"]   
tweetsPerPage = 25 # Must be smaller than dataset size
timeRemaining = 1200 # Time left in seconds

s = tb.createSession(fileName, False)
out = widgets.Output()

def resetDisplay(b = None):
    out.clear_output(True)
    with out:
        display(bar) 
    
def randomSample(b):
    if(s.currentSet.size < s.length):
        s.back()
    s.simpleRandomSample(tweetsPerPage)
    getTweets(b)
    
def searchKeyword(b):
    if(s.currentSet.size < s.length):
        s.back()
    if searchBox.value == "":
        s.simpleRandomSample(tweetsPerPage)
    else:
        s.searchKeyword(searchBox.value.split(' '))
    getTweets(b)
    
    
def getTweets(b):
    if timeRemaining <= 0:
        return
    resetDisplay()
    htmlText = "<style>table, th, td {border: 1px solid black;} th, td {padding: 10px;}</style>"
    htmlText += "<table> <tr> "
    for header in colHeaders:
        htmlText += " <th> " + header + " </th> "
    dataSet = s.getCurrentSubset()
    for i in range(min(len(dataSet), tweetsPerPage)):
        htmlText += " <tr> "
        for j in colHeaders:
            htmlText += " <td>" + str(dataSet.iloc[i].loc[j]) + "</td> "
        htmlText += " </tr> "
    table = widgets.HTML(value=htmlText + " </table>")
    with out:
        display(table)
        
def timer(timerWidget):
    global timeRemaining
    while timeRemaining > 0:
        timeRemaining -= 1
        if timeRemaining == 0:
            timerWidget.value = "Time remaining: 00:00"
            return
        minsLeft = timeRemaining // 60
        secsLeft = timeRemaining % 60
        if minsLeft < 10:
            minsLeft = "0" + str(minsLeft)
        if secsLeft < 10:
            secsLeft = "0" + str(secsLeft)
        timerWidget.value = "Time remaining: " + str(minsLeft) + ":" + str(secsLeft)
        time.sleep(1)

#Search Bar
searchBox = widgets.Text(value='', placeholder='Keyword', description='Search for:')
searchButton = widgets.Button(description='Get Tweets', 
                tooltip='Search for tweets containing all words in the current input. If no input is provided, a random sample is given.')
searchBar = widgets.HBox([searchBox, searchButton])
searchButton.on_click(searchKeyword)   

# Timer
timerWidget = widgets.HTML(value = "Time remaining:", )
bar = widgets.VBox([timerWidget, searchBar])

searchButton.on_click(searchKeyword)
thread = threading.Thread(target=timer, args=(timerWidget,))
thread.start() 

with out:
    display(bar)
    
out

Output()